In [1]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torch
from torchvision import transforms
import import_ipynb
from config import batch_size, rootDir, device, lr, epochs, base_output
from data import AFAD
from model import MTL
from glob import glob
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

importing Jupyter notebook from config.ipynb
importing Jupyter notebook from data.ipynb
importing Jupyter notebook from model.ipynb


In [2]:
def save_loss_plot(out_dir, train_loss, val_loss):
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    train_ax.plot(train_loss, color='tab:blue')
    train_ax.set_xlabel('iterations')
    train_ax.set_ylabel('train loss')
    valid_ax.plot(val_loss, color='tab:red')
    valid_ax.set_xlabel('iterations')
    valid_ax.set_ylabel('validation loss')
    figure_1.savefig(f"{out_dir}/train_loss.png")
    figure_2.savefig(f"{out_dir}/valid_loss.png")
    print('SAVING PLOTS COMPLETE...')
    
    plt.close('all')

In [3]:
images_path = []
ages = []
genders = []

for age in os.listdir(rootDir):
    age_files = os.path.join(rootDir, age)
    if os.path.isdir(age_files):
        for gender in os.listdir(age_files):
            gender_files = os.path.join(age_files, gender)
            if os.path.isdir(gender_files):
                for img in os.listdir(gender_files):
                    image_path = os.path.join(gender_files, img)
                    images_path.append(image_path)
                    ages.append(int(age))
                    genders.append(int(gender[2])-1)

In [4]:
train_paths, val_paths = train_test_split(
    images_path, test_size=0.2, random_state=10
)
train_ages, val_ages = train_test_split(
    ages, test_size=0.2, random_state=10
)
train_genders, val_genders = train_test_split(
    genders, test_size=0.2, random_state=10
)
train_dataloader = DataLoader(
    AFAD(train_paths, train_ages, train_genders), shuffle=True, batch_size=batch_size
)
val_dataloader = DataLoader(
    AFAD(val_paths, val_ages, val_genders), shuffle=False, batch_size=batch_size
)

In [5]:
model = MTL().to(device=device)
# opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.09)
opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.02, amsgrad=False)
# gender_loss = nn.BCELoss() # output必須要Sigmoid
gender_loss = nn.CrossEntropyLoss()
age_loss = nn.L1Loss()
Sig = nn.Sigmoid()

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
H = {"train_loss": [], "valid_loss": []}

In [7]:
model.load_state_dict(torch.load('output/bestmodel1-472.pth'))

<All keys matched successfully>

In [ ]:
best_val_loss = 99999
for epoch in range(epochs):
    model.train()
    training_loss = 0
    for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        inputs = data["image"].to(device=device)
        age_label = data["age"].to(device=device)
        gender_label = data["gender"].to(device=device)
        
        opt.zero_grad()
        
        age_output, gender_output = model(inputs)
#         loss_1 = gender_loss(Sig(gender_output), gender_label.unsqueeze(1).float())
        loss_1 = gender_loss(gender_output, gender_label)
        loss_2 = age_loss(age_output, age_label.unsqueeze(1).float())
        loss = loss_1 + loss_2
        loss.backward()
        opt.step()
        training_loss += loss.item()
    avgTrainLoss = training_loss/ len(train_dataloader)
    print(  
            'epoch: {} train_epoch_loss {}'.format(
                epoch+1,
                avgTrainLoss,
            )
        )
    H["train_loss"].append(avgTrainLoss)
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        for i, data in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
            inputs = data["image"].to(device=device)
            age_label = data["age"].to(device=device)
            gender_label = data["gender"].to(device=device)

            opt.zero_grad()

            age_output, gender_output = model(inputs)
#             loss_1 = gender_loss(Sig(gender_output), gender_label.unsqueeze(1).float())
            loss_1 = gender_loss(gender_output, gender_label)
            loss_2 = age_loss(age_output, age_label.unsqueeze(1).float())
            loss = loss_1 + loss_2
            valid_loss += loss.item()
        avgValidLoss = valid_loss / len(val_dataloader)
        print(  
            'epoch: {} val_epoch_loss {}'.format(
                epoch+1,
                avgValidLoss,
            )
        )
        H["valid_loss"].append(avgValidLoss)
        
        if avgValidLoss < best_val_loss:
            best_val_loss = avgValidLoss
            torch.save(model.state_dict(), f'{base_output}/bestmodel.pth')
        save_loss_plot(
                base_output, H["train_loss"],  H["valid_loss"]
        )
        torch.save(model.state_dict(), f'{base_output}/model.pth')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [06:31<00:00,  8.32s/it]


epoch: 1 train_epoch_loss 2.546395905474399


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [02:34<00:00, 12.84s/it]


epoch: 1 val_epoch_loss 3.692653755346934
SAVING PLOTS COMPLETE...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [01:17<00:00,  1.65s/it]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [01:12<00:00,  1.53s/it]


epoch: 43 train_epoch_loss 0.7604218125343323


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:13<00:00,  1.14s/it]


epoch: 43 val_epoch_loss 4.098646382490794
SAVING PLOTS COMPLETE...


  6%|████████████▊                                                                                                                                                                                           | 3/47 [00:04<01:10,  1.60s/it]